# Brian Replication of data_model.py in a python notebook

In [157]:
import pandas as pd
import logging
import dateutil
from dateutil import parser
pd.set_option('display.max_columns', 500)

In [32]:
logging.basicConfig(level=logging.INFO)

# Code Broken Down

In [244]:
df = pd.read_csv("cleaned/all_quarters_merged_TEST.csv")

/Users/briangoggin/anaconda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (1,2,6,12,16,17,21,22,28,29,30,31,32,33,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,51,52,53,61,79,88,91,92,93,94,96,97,98,99,100,101,103,113,114,115,116,117,118,119,120,121,122,123,140,142,143,144,145,146) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [245]:
# keep only variables of interest
list = ['address', 'apn', 'best_date', 'best_stat','firstfiled', 'report_quarter', 'report_year', 'unitsnet', 'dbi_permit', 'x', 'y']
df = df[list]

In [246]:
#consolidate status categories. Start with 3 for now
def status_function(value):
    if value['best_stat']=="CONSTRUCTION":
        field = 'Under Construction'
    elif (value['best_stat']=='BP APPROVED') | (value['best_stat']=='BP ISSUED') | (value['best_stat']=='BP REINSTATED'):
        field = 'Building Permit Approved'
    else:
        field = 'Proposed'
    return field
        
df['status']=df.apply(status_function, axis=1)

In [255]:
df.head()
#df[['apn']==4172053]

,address,apn,best_date,best_stat,firstfiled,report_quarter,report_year,unitsnet,dbi_permit,x,y,status
0,1179 TENNESSEE ST,4172053,2009-11-03,BP REINSTATED,1999-02-11 08:00:00,1,2012,8.0,9902819,37.756751,-122.388969,Building Permit Approved
1,2026 LOMBARD ST,0492025,2007-04-11,BP ISSUED,2002-06-11 07:00:00,1,2012,0.0,200606023107,37.800127,-122.434940,Building Permit Approved
2,580 OAK PARK DR,2676030,2007-11-28,BP ISSUED,2002-12-04 08:00:00,1,2012,1.0,200212042761,37.756322,-122.460196,Building Permit Approved
3,2525 MISSION ST,3615026,2009-08-26,BP ISSUED,2003-02-21 08:00:00,1,2012,0.0,200302218059,37.756561,-122.418695,Building Permit Approved
4,244 GRANADA AV,6942039,2008-10-09,BP ISSUED,2003-03-25 08:00:00,1,2012,1.0,200701051057,37.722873,-122.456990,Building Permit Approved


In [268]:
df['apn'][2118].str.zfill(7)

AttributeError: 'str' object has no attribute 'str'

In [267]:
df[df['address']=='526 HICKORY ST']

,address,apn,best_date,best_stat,firstfiled,report_quarter,report_year,unitsnet,dbi_permit,x,y,status
2118,526 HICKORY ST,829023,2012-09-20,BP FILED,2012-09-20,3,2012,1.0,2.01209E+11,-122.428274,37.774782,Proposed
2933,526 HICKORY ST,0829023,2012-09-20,BP FILED,2012-09-20,4,2012,1.0,201209200264,37.774782,-122.428274,Proposed
3662,526 HICKORY ST,0829023,2012-09-20,BP FILED,2012-09-20,1,2013,1.0,201209200264,37.774782,-122.428274,Proposed
4091,526 HICKORY ST,0829023,2012-09-20,BP FILED,2012-09-20,2,2013,1.0,201209200264,37.774782,-122.428274,Proposed
5115,526 HICKORY ST,0829023,2012-09-20,BP FILED,2012-09-20,3,2013,1.0,201209200264,37.774782,-122.428274,Proposed
6068,526 HICKORY ST,0829023,2013-11-22,BP ISSUED,2012-09-20,4,2013,1.0,201209200264,37.774782,-122.428274,Building Permit Approved
6889,526 HICKORY ST,0829023,2013-11-22,BP ISSUED,2012-09-20,1,2014,1.0,201209200264,37.774782,-122.428274,Building Permit Approved
7752,526 HICKORY ST,829023,2013-11-22,BP ISSUED,2012-09-20,2,2014,1.0,201209200264,37.774782,-122.428274,Building Permit Approved
8546,526 HICKORY ST,0829023,2014-09-16,CONSTRUCTION,2012-09-20,3,2014,1.0,201209200264,37.774782,-122.428274,Under Construction
9781,526 HICKORY ST,0829023,2015-01-21,CONSTRUCTION,NaN,4,2014,1.0,NaN,37.774782,-122.428274,Under Construction


In [37]:
gb = df.groupby(['apn', 'address'])

In [38]:
gb.groups

{('0642036', '1700 CALIFORNIA ST'): Int64Index([14917], dtype='int64'),
 ('1407017',
  '2500 CLEMENT ST'): Int64Index([3712, 4137, 5162, 5929, 6779, 7778, 8596, 9271, 10056], dtype='int64'),
 ('478003', '2815 POLK ST'): Int64Index([10080], dtype='int64'),
 ('1407046',
  '6423 CALIFORNIA ST'): Int64Index([7677, 8860, 9611, 10267, 11145, 12985, 14373, 15582], dtype='int64'),
 ('0828012',
  '735 FELL ST'): Int64Index([576, 1261, 2334, 3023, 4348, 5388], dtype='int64'),
 ('0811002',
  '101 POLK ST'): Int64Index([772, 2902, 3633, 4484, 5094, 6043, 6864, 8529, 9064, 11104, 12111,
             13309],
            dtype='int64'),
 ('3750082',
  '667 FOLSOM ST 120 HAWTHORNE ST 126 HAWTHORNE ST'): Int64Index([12095, 13289, 15606], dtype='int64'),
 ('0829040', '526 HICKORY ST'): Int64Index([11461], dtype='int64'),
 ('1811001',
  '3701 JUDAH ST'): Int64Index([11746, 13042, 14111, 15340], dtype='int64'),
 ('0937028',
  '2419 2421 &AMP; 2435 LOMBARD ST'): Int64Index([2847, 3666, 4083, 5118, 6071, 68

In [243]:
group_df = gb.get_group(('829040','526 HICKORY ST'))

KeyError: ('829040', '526 HICKORY ST')

In [227]:
group_df = group_df.sort('best_date')
group_df.reset_index(drop=True, inplace=True)

/Users/briangoggin/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


In [228]:
#group_df['firstfiled'].fillna(value='', inplace = True)

In [229]:
group_df

,address,apn,best_date,best_stat,firstfiled,report_quarter,report_year,unitsnet,dbi_permit,x,y,status
0,75 MARS ST,2653016,2012-03-08,BP ISSUED,2010-06-15 07:00:00,1,2012,1.0,201006154512,37.761012,-122.444483,Building Permit Approved
1,75 MARS ST,2653016,2012-03-08,BP ISSUED,2010-06-15 00:00:00,2,2012,1.0,NaN,37.761012,-122.444483,Building Permit Approved
2,75 MARS ST,2653016,2012-03-08,BP ISSUED,2010-06-15,3,2012,1.0,2.01006E+11,-122.444483,37.761012,Building Permit Approved
3,75 MARS ST,2653016,2012-03-08,BP ISSUED,2010-06-15,4,2012,1.0,201006154512,37.761012,-122.444483,Building Permit Approved
4,75 MARS ST,2653016,2012-03-08,BP ISSUED,2010-06-15,1,2013,1.0,201006154512,37.761012,-122.444483,Building Permit Approved
5,75 MARS ST,2653016,2012-03-08,BP ISSUED,2010-06-15,2,2013,1.0,201006154512,37.761012,-122.444483,Building Permit Approved
6,75 MARS ST,2653016,2012-03-08,BP ISSUED,2010-06-15,3,2013,1.0,201006154512,37.761012,-122.444483,Building Permit Approved
7,75 MARS ST,2653016,2013-10-18,BP FILED,2010-09-01,4,2013,1.0,201310189655,37.761012,-122.444483,Proposed
8,75 MARS ST,2653016,2014-03-26,BP APPROVED,2010-09-01,1,2014,1.0,201310189655,37.761012,-122.444483,Building Permit Approved
9,75 MARS ST,2653016,2014-04-23,BP ISSUED,2010-09-01,2,2014,1.0,201310189655,37.761012,-122.444483,Building Permit Approved


In [231]:
#Identify earliest "firstfiled" date
firstfiled = ''
for index, row in group_df.iterrows():
    if pd.isnull(row['firstfiled']):
        continue
    else:
        if len(firstfiled) == 0:
            firstfiled = row['firstfiled']
        else:
            continue

In [232]:
#Identify earliest best date
earliest_BD = ''
for index, row in group_df.iterrows():
    if pd.isnull(row['best_date']):
        continue
    else:
        if len(earliest_BD) == 0:
            earliest_BD = row['best_date']
        else:
            continue

In [233]:
firstfiled

'2010-06-15 07:00:00'

In [234]:
earliest_BD

'2012-03-08'

In [235]:
# Finalize first date variable (minimum of earliest best_date and firstfiled)
if pd.isnull(firstfiled) & pd.notnull(earliest_BD):
    first_date = earliest_BD
elif pd.isnull(earliest_BD) & pd.notnull(firstfiled):
    first_date = firstfiled
elif pd.notnull(firstfiled) & pd.notnull(earliest_BD):
    first_date = min(firstfiled, earliest_BD)

In [236]:
first_date

'2010-06-15 07:00:00'

In [237]:
# initiate variables. Groups without these dates are blank for these variables.
BP_date = ''
con_date = ''

#Identify first date for all status categories
m=0
for index, row in group_df.iterrows():
    if m == 0:
        status_previous = 'blah'
    if row['status']=='Building Permit Approved':
        if index == 0:
            BP_date = row['best_date']
        elif index !=0:
            if status_previous =='Building Permit Approved':
                BP_date=BP_date
            else:
                BP_date = row['best_date']
    elif row['status']=='Under Construction':
        if index == 0:
            con_date = row['best_date']
        elif index !=0:
            if status_previous =='Under Construction':
                con_date=con_date
            else:
                con_date = row['best_date']
    status_previous = row['status']
    m=m+1

In [238]:
print (BP_date, con_date)

2014-03-26 


In [191]:
last_row = group_df.tail(1).copy() #most recent row

In [192]:
last_row

,address,apn,best_date,best_stat,dbi_permit,report_quarter,report_year,unitsnet,x,y
7532,4132 THIRD STREET,5260002,2014-07-28,CONSTRUCTION,200707055953,3,2014,7.0,37.740114,-122.389089


In [193]:
last_row['latest_project_record_date'] = last_row.best_date
last_row['first_project_record_date'] = group_df.iloc[0].best_date
last_row['latest_project_status'] = last_row.best_stat

In [194]:
last_row

,address,apn,best_date,best_stat,dbi_permit,report_quarter,report_year,unitsnet,x,y,latest_project_record_date,first_project_record_date,latest_project_status
7532,4132 THIRD STREET,5260002,2014-07-28,CONSTRUCTION,200707055953,3,2014,7.0,37.740114,-122.389089,2014-07-28,2013-08-08,CONSTRUCTION


In [195]:
## Store a parseable list of all the project states and the dates those states were reported
last_row['project_dates'] = str(tuple(group_df.best_date))
last_row['project_statuses'] = str(tuple(group_df.best_stat))

In [196]:
last_row

,address,apn,best_date,best_stat,dbi_permit,report_quarter,report_year,unitsnet,x,y,latest_project_record_date,first_project_record_date,latest_project_status,project_dates,project_statuses
7532,4132 THIRD STREET,5260002,2014-07-28,CONSTRUCTION,200707055953,3,2014,7.0,37.740114,-122.389089,2014-07-28,2013-08-08,CONSTRUCTION,"('2013-08-08', '2013-08-08', '2014-02-26', '20...","('BP ISSUED', 'BP ISSUED', 'CONSTRUCTION', 'CO..."


In [197]:
## Store the project duration in days
latest = last_row.latest_project_record_date.iloc[0]
first = last_row.first_project_record_date.iloc[0]


In [198]:
if not (pd.isnull(latest) or pd.isnull(first)):
    last_row['project_duration_days'] = (dateutil.parser.parse(latest) - dateutil.parser.parse(first)).days

In [199]:
last_row

,address,apn,best_date,best_stat,dbi_permit,report_quarter,report_year,unitsnet,x,y,latest_project_record_date,first_project_record_date,latest_project_status,project_dates,project_statuses,project_duration_days
7532,4132 THIRD STREET,5260002,2014-07-28,CONSTRUCTION,200707055953,3,2014,7.0,37.740114,-122.389089,2014-07-28,2013-08-08,CONSTRUCTION,"('2013-08-08', '2013-08-08', '2014-02-26', '20...","('BP ISSUED', 'BP ISSUED', 'CONSTRUCTION', 'CO...",354


# Actual Code

In [241]:
def convert_to_one_record_per_project(df):
    """
    Group the dataset by (apn, address) and then emit one row per (apn, address) pair.
    best_date and best_stat will be converted into arrays.
    All other attributes will be taken from the record with the most recent `best_date` attribute.
    """
    gb = df.groupby(['apn', 'address'])
    for k in gb.groups:
        group_df = gb.get_group(k)
        group_df = group_df.sort('best_date')
        last_row = group_df.tail(1).copy()
        
        #Identify earliest "firstfiled" date
        firstfiled = ''
        for index, row in group_df.iterrows():
            if pd.isnull(row['firstfiled']):
                continue
            else:
                if len(firstfiled) == 0:
                    firstfiled = row['firstfiled']
                else:
                    continue
        
        #Identify earliest best date
        earliest_BD = ''
        for index, row in group_df.iterrows():
            if pd.isnull(row['best_date']):
                continue
            else:
                if len(earliest_BD) == 0:
                    earliest_BD = row['best_date']
                else:
                    continue
        
        # Finalize first date variable (minimum of earliest best_date and firstfiled)
        if pd.isnull(firstfiled) & pd.notnull(earliest_BD):
            first_date = earliest_BD
        elif pd.isnull(earliest_BD) & pd.notnull(firstfiled):
            first_date = firstfiled
        elif pd.notnull(firstfiled) & pd.notnull(earliest_BD):
            first_date = min(firstfiled, earliest_BD)
            
        # initiate variables. Groups without these dates are blank for these variables.
        BP_date = ''
        con_date = ''

        #Identify first date for all status categories
        m=0
        for index, row in group_df.iterrows():
            if m == 0:
                status_previous = 'blah'
            if row['status']=='Building Permit Approved':
                if index == 0:
                    BP_date = row['best_date']
                elif index !=0:
                    if status_previous =='Building Permit Approved':
                        BP_date=BP_date
                    else:
                        BP_date = row['best_date']
            elif row['status']=='Under Construction':
                if index == 0:
                    con_date = row['best_date']
                elif index !=0:
                    if status_previous =='Under Construction':
                        con_date=con_date
                    else:
                        con_date = row['best_date']
            status_previous = row['status']
            m=m+1
        
        last_row['BP_date'] = BP_date
        last_row['con_date'] = con_date
        last_row['first_date']=first_date
        last_row['latest_project_record_date'] = last_row.best_date
        last_row['first_project_record_date'] = group_df.iloc[0].best_date
        last_row['latest_project_status'] = last_row.best_stat

        ## Store a parseable list of all the project states and the dates those states were reported
        last_row['project_dates'] = str(tuple(group_df.best_date))
        last_row['project_statuses'] = str(tuple(group_df.best_stat))

        ## Store the project duration in days
        latest = last_row.latest_project_record_date.iloc[0]
        first = last_row.first_project_record_date.iloc[0]
        if not (pd.isnull(latest) or pd.isnull(first)):
            last_row['project_duration_days'] = (dateutil.parser.parse(latest) - dateutil.parser.parse(first)).days

        yield last_row


def main():
    df = pd.read_csv("cleaned/all_quarters_merged_TEST.csv")
    #consolidate status categories. Start with 3 for now
    def status_function(value):
        if value['best_stat']=="CONSTRUCTION":
            field = 'Under Construction'
        elif (value['best_stat']=='BP APPROVED') | (value['best_stat']=='BP ISSUED') | (value['best_stat']=='BP REINSTATED'):
            field = 'Building Permit Approved'
        else:
            field = 'Proposed'
        return field

    df['status']=df.apply(status_function, axis=1)
    
    #keep only variables we want
    list = ['address', 'apn', 'best_date', 'best_stat','status', 'firstfiled', 'report_quarter', 'report_year', 'unitsnet', 'dbi_permit', 'x', 'y']
    df = df[list]
    new_df = pd.concat(convert_to_one_record_per_project(df))
    logging.info("Writing output ({} rows, {} cols) to data/cleaned/all_quarters__one_record_per_project.csv".format(*new_df.shape))
    new_df.to_csv("cleaned/all_quarters__one_record_per_project_BRIAN.csv")

In [242]:
main()

/Users/briangoggin/anaconda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2831: DtypeWarning: Columns (1,2,6,12,16,17,21,22,28,29,30,31,32,33,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,51,52,53,61,79,88,91,92,93,94,96,97,98,99,100,101,103,113,114,115,116,117,118,119,120,121,122,123,140,142,143,144,145,146) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):
/Users/briangoggin/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:10: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
